In [2]:
import pathlib

In [12]:
SMS_SPAM_ZIP='https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip'
YOUTUBE_SPAM_ZIP='https://archive.ics.uci.edu/ml/machine-learning-databases/00380/YouTube-Spam-Collection-v1.zip'

In [46]:
BASE_DIR=pathlib.Path().resolve().parent
DATASET_DIR=BASE_DIR / "datasets"
ZIPS_DIR = DATASET_DIR / "zips"
ZIPS_DIR.mkdir(exist_ok=True, parents=True)

SMS_SPAM_ZIP_PATH = ZIPS_DIR / 'sms_spam.zip'
YOUTUBE_SPAM_ZIP_PATH = ZIPS_DIR / 'youtube_spam.zip'

EXPORT_DIR=DATASET_DIR / "exports"
EXPORT_DIR.mkdir(exist_ok=True, parents=True)
SPAM_DATASET_PATH=EXPORT_DIR/"spam_dataset.csv"

In [10]:
!curl -i $SMS_SPAM_ZIP -o $SMS_SPAM_ZIP_PATH

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  198k  100  198k    0     0  16388      0  0:00:12  0:00:12 --:--:-- 49565


In [13]:
!curl -i $YOUTUBE_SPAM_ZIP -o $YOUTUBE_SPAM_ZIP_PATH

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  159k  100  159k    0     0  41609      0  0:00:03  0:00:03 --:--:-- 41598


In [14]:
SPAM_CLASSIFIER_DIR = DATASET_DIR / "spam-classifier"
SMS_SPAM_DIR = DATASET_DIR / "sms-spam"
YOUTUBE_SPAM_DIR = DATASET_DIR / "youtube-spam"

SMS_SPAM_DIR.mkdir(exist_ok=True, parents=True)
YOUTUBE_SPAM_DIR.mkdir(exist_ok=True, parents=True)


In [16]:
!unzip -o $SMS_SPAM_ZIP_PATH -d $SMS_SPAM_DIR
!unzip -o $YOUTUBE_SPAM_ZIP_PATH -d $YOUTUBE_SPAM_DIR

Archive:  /Users/jerome/Downloads/jupyter_dir/Tutorial/AI_as_API/datasets/zips/sms_spam.zip
warning [/Users/jerome/Downloads/jupyter_dir/Tutorial/AI_as_API/datasets/zips/sms_spam.zip]:  332 extra bytes at beginning or within zipfile
  (attempting to process anyway)
  inflating: /Users/jerome/Downloads/jupyter_dir/Tutorial/AI_as_API/datasets/sms-spam/SMSSpamCollection  
  inflating: /Users/jerome/Downloads/jupyter_dir/Tutorial/AI_as_API/datasets/sms-spam/readme  
Archive:  /Users/jerome/Downloads/jupyter_dir/Tutorial/AI_as_API/datasets/zips/youtube_spam.zip
warning [/Users/jerome/Downloads/jupyter_dir/Tutorial/AI_as_API/datasets/zips/youtube_spam.zip]:  332 extra bytes at beginning or within zipfile
  (attempting to process anyway)
  inflating: /Users/jerome/Downloads/jupyter_dir/Tutorial/AI_as_API/datasets/youtube-spam/Youtube01-Psy.csv  
   creating: /Users/jerome/Downloads/jupyter_dir/Tutorial/AI_as_API/datasets/youtube-spam/__MACOSX/
  inflating: /Users/jerome/Downloads/jupyter_dir/

In [26]:
import pandas as pd

In [42]:
SMS_SPAM_DATA_PATH=SMS_SPAM_DIR/'SMSSpamCollection'
# SMS_SPAM_DATA_PATH.read_text()
# for path in SMS_SPAM_DIR.glob('*'):
#     print(path)
#     try:
#         print(path.read_text())
#     except:
#         pass
sms_df=pd.read_csv(SMS_SPAM_DATA_PATH, sep='\t', header=None, names=['label', 'text'])
sms_df['source']=SMS_SPAM_DATA_PATH.name
sms_df.head()

,label,text,source
0,ham,"Go until jurong point, crazy.. Available only ...",SMSSpamCollection
1,ham,Ok lar... Joking wif u oni...,SMSSpamCollection
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,SMSSpamCollection
3,ham,U dun say so early hor... U c already then say...,SMSSpamCollection
4,ham,"Nah I don't think he goes to usf, he lives aro...",SMSSpamCollection


In [43]:
yt_dfs=[]
for path in YOUTUBE_SPAM_DIR.glob('*csv'):
    print(path)
    try:
        df=pd.read_csv(path)
        df.rename(columns={'CLASS': 'raw_label', 'CONTENT':'text'}, inplace=True)
        df['label']=df['raw_label'].apply(lambda x: 'spam' if str(x)=='1' else 'ham')
        df['source']=path.name
        yt_dfs.append(df[['label', 'text', 'source']])
#         print(yt_df.head())
    except:
        pass
    print('='*30)
yt_df=pd.concat(yt_dfs)
yt_df.head()

/Users/jerome/Downloads/jupyter_dir/Tutorial/AI_as_API/datasets/youtube-spam/Youtube03-LMFAO.csv
/Users/jerome/Downloads/jupyter_dir/Tutorial/AI_as_API/datasets/youtube-spam/Youtube04-Eminem.csv
/Users/jerome/Downloads/jupyter_dir/Tutorial/AI_as_API/datasets/youtube-spam/Youtube05-Shakira.csv
/Users/jerome/Downloads/jupyter_dir/Tutorial/AI_as_API/datasets/youtube-spam/Youtube02-KatyPerry.csv
/Users/jerome/Downloads/jupyter_dir/Tutorial/AI_as_API/datasets/youtube-spam/Youtube01-Psy.csv


,label,text,source
0,ham,"<a href=""http://www.youtube.com/watch?v=KQ6zr6...",Youtube03-LMFAO.csv
1,ham,wierd but funny﻿,Youtube03-LMFAO.csv
2,spam,"Hey guys, I&#39;m a human.<br /><br /><br />Bu...",Youtube03-LMFAO.csv
3,ham,Party Rock....lol...who wants to shuffle!!!﻿,Youtube03-LMFAO.csv
4,ham,Party rock﻿,Youtube03-LMFAO.csv


In [44]:
df=pd.concat([sms_df, yt_df])
df.head()

,label,text,source
0,ham,"Go until jurong point, crazy.. Available only ...",SMSSpamCollection
1,ham,Ok lar... Joking wif u oni...,SMSSpamCollection
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,SMSSpamCollection
3,ham,U dun say so early hor... U c already then say...,SMSSpamCollection
4,ham,"Nah I don't think he goes to usf, he lives aro...",SMSSpamCollection


In [47]:
df.to_csv(SPAM_DATASET_PATH, index=None)